In [21]:
import numpy as np
from PIL import Image, ImageOps, ImageEnhance,ImageFilter
import matplotlib.pyplot as plt
import random
import os

In [22]:
from PIL import Image
import numpy as np

class ImageLoader:
    def __init__(self, path = None, img  =None):
        self.img = img
        self.path = path
        
    def load_Image(self,path):
        
        self.img = Image.open(path)
        self.image_array = np.array(self.img)
        return self.image_array  # fixed typo: was returning self.Image
        
    def save_Image(self,array, filename):
        img_to_file = Image.fromarray(array.astype(np.uint8))
        img_to_file.save(filename)

In [23]:
class Processing:
    def GreyCode(self,img):  #to bring image in one color
        # image is always taken an array, because when we load image , we load it in the form of array
        # formula to make a grey_code image
        image_array = np.array(img)
        gray = 0.2989 * image_array[:, :, 0] + \
               0.5870 * image_array[:, :, 1] + \
               0.1140 * image_array[:, :, 2]
        return gray.astype(np.uint8)  # nint8 keeps the image value of the pixel in 0-255

    def normalize_contract(self , img): #To adjust the brightness and contrast of an image so that its
                                # pixel values span the full intensity range (0–255), making the image clearer,
                                #more visually balanced, and easier to analyze or edit.
        image_array = img.astype(np.float32)
        normalize = (image_array - image_array.min()) * (255.0 / (image_array.max() - image_array.min()))
        return normalize.astype(np.uint8)

    def standarize(self,img): #Standard deviation tells you how “spread out” the pixel values are. Low std = flat/low contrast )(grey), high std = lots of detail/contrast.
        image_array = img.astype(np.float32)
        mean = image_array.mean()
        std = image_array.std()
        standerized = (image_array - mean) / std
        return standerized

    def resize(self,img,new_shape):
        # change the shape of the image
        return img.resize(new_shape)
        
    def flaten(self,img):  # make a 2dor 3d image in a single array
        image_array = np.array(img)
        return image_array.flatten()  # it flat the image in 1D
        
    def seperate_Channel(self,img):
        image_array = np.array(img)
        R = image_array[:,:,0]
        G = image_array[:,:,1]
        B = image_array[:,:,2]
        return R,G,B

    def check_missings(self, img, percent = 0.1):  # used to set the missing values and make it nan like none
        image_array = np.array(img).astype(np.float32)
        total_pixel = image_array.size
        missing_number = int(total_pixel * percent)
        indexes = np.random.choice(total_pixel, missing_number, replace=False)
        flat  = image_array.flatten()
        flat[indexes] = np.nan
        return flat.reshape(image_array.shape)

    def fill_missing(self,img,Method = 'mean'):  # fill the missing element
        image_array = np.array(img).astype(np.float32)
        if (Method == 'mean'): # fill the element with mean of all the nan pixel and store them , for balance brighness 
            mean_val = np.nanmean(image_array)
            image_array[np.isnan(image_array)] = mean_val
        elif (Method == 'zero'):
            image_array[np.isnan(image_array)] = 0  # fill with zero( black)
        elif (Method == 'median'):
            median_val = np.nanmedian(image_array)
            image_array[np.isnan(image_array)] = median_val  # median value, preserves brightness more neutrally
        return image_array.astype(np.uint8)
    

In [24]:
class Transformation:
    
    def Invert_image(self,img):
        return ImageOps.invert(img)

    def adjust_brighness(self,img):
        try:
            brighnes = float(input("Enter the Brightness (>1 for brighness , <1 for darkness): "))
        except:
            print('Invalid , use 1 as default')
            brighnes = 1.0
        enhancer = ImageEnhance.Brightness(img) # class object
        return enhancer.enhance(brighnes)  # instance function to change brighness

    def acjust_constract(self,img):
        try:
            contract_value = float(input("Enter the value of constrast  (>1 for higher) and (<1 for lower)"))
        except:
            print("Invalid , Printing the default value")
            contract_value= 1.0
        enhance = ImageEnhance.Contrast(img)
        return enhance.enhance(contract_value)

    def Rotate(self, img):
        try:
            angle = float(input("Enter the angle of rotation (0 to 360): "))
        except:
            print("Invalid, Default set as 0 degree")
            angle =0.0
        return img.rotate(angle, expand = True)

    def Flip_Horizontal(self, img):
        return ImageOps.mirror(img)

    def Flip_Vertical(self, img):
        return ImageOps.flip(img)  

    def Crop(self,img):
        try:
            x1 = int(input("Enter x1: "))
            y1 = int(input("Enter y1: "))
            x2 = int(input("Enter x2: "))
            y2 = int(input("Enter y2: "))
        except:
            print("Invalid , Returning full image")
            x1,y1,x2y2 = 0,0,img.width ,img.height
        return img.crop((x1,y1,x2,y2))
    

In [35]:
class Analyzer:
    def mean(self , img):
        img_array = np.array(img, dtype = np.float32)
        return np.mean(img_array)

    def variance(self , img):
        img_array = np.array(img, dtype = np.float32)
        return np.var(img_array)

    def std(self,img):
        img_array = np.array(img, dtype=np.float32)
        return np.std(img_array)

    def min_max_Pixel(self,img):
        img_array = np.array(img, dtype=np.float32)
        return np.min(img_array), np.max(img_array)
        
    def correlation(self , img):
        img_array = np.array(img, dtype=np.float32)
        if (img_array.shape != 3 or img_array.shape[2] != 3):
            print("Corelation only exist with RGB")
            return None
        
        R = image_array[:, :,0].flatten()
        G = image_array[:, :,1].flatten()
        B = image_array[:, :,2].flatten()

        coRel_matrix = np.corrcoef([R,G,B])
        return coRel_matrix

    def Histogram(self,img,bins = 256):
        img_array = np.array(img , dtype = np.float32)
        if (img_array.ndim == 2):
            plt.hist(img_array.flatten(), bins = bins , color = 'grey')
            plt.title("GreyScale Histogram")
            plt.xlabel('Pixel')
            plt.ylabel('Frequency')
            plt.show()

        elif (img_array.ndim == 3 or img_array.shape[2] == 3):
            color = ['r' , 'b' , 'g']
            for i,c in enumerate(color):
                plt.hist(img_array[:,:,i].flatten() , bins = bins , color = c , label = c.upper(), alpha = 0.5)
            plt.title('Colored Histogram')
            plt.xlabel('Pixel')
            plt.ylabel('Frequency')
            plt.show() 

In [36]:
class AI_Features:
    
    def blur(self, img):
        return img.filter(ImageFilter.BLUR)

    def sharpen(self,img):
        return img.filter(ImageFilter.SHARPEN)

    def smooth(self,img):
        return img.filter(ImageFilter.SMOOTH)

    def egde_detection(self,img):
        pass
        

In [41]:
class Exceptoion_Handling:
    def check_file(self , path):
        if os.path.isfile(path):
            print(f"Yes, file exist: {path}")
            return True
        else:
            print(f'The file does not exist in {path}')
            return False

In [ ]:
#### import matplotlib.pyplot as plt

def show_image(img_array, title="Image", cmap=None):
    plt.figure(figsize=(5,5))
    if cmap:
        plt.imshow(img_array, cmap=cmap)
    else:
        plt.imshow(img_array)
    plt.title(title)
    plt.axis("off")
    plt.show()

def main():
    loader = ImageLoader()
    file_path = input("Enter the path of the image to load: ")
    if not os.path.isfile(file_path):
        print("❌ File does not exist.")
        return

    img_array = loader.load_Image(file_path)
    img = Image.open(file_path) 
    print("✅ Image loaded successfully!")

    proc = Processing()

    gray_img = proc.GreyCode(img)
    normalized_img = proc.normalize_contract(gray_img)
    standardized_img = proc.standarize(gray_img)

    flat_array = proc.flaten(gray_img)
    R, G, B = proc.seperate_Channel(img)

    new_w = int(input("Enter new width for resizing: "))
    new_h = int(input("Enter new height for resizing: "))
    resized_img = proc.resize(img, (new_w, new_h))

    show_image(np.array(resized_img), title="Resized Image (Final Result)")


    analyzer = Analyzer()
    print("Mean pixel value:", analyzer.mean(img))
    print("Variance:", analyzer.variance(img))
    print("Standard deviation:", analyzer.std(img))
    print("Min & Max pixels:", analyzer.min_max_Pixel(img))

    if img.mode == 'RGB':
        print("Channel correlation:\n", analyzer.correlation(img))

    analyzer.Histogram(img)

    # ------------------- TRANSFORMATION------------------
    trans = Transformation()

    inverted_img = trans.Invert_image(img)
    show_image(np.array(inverted_img), title="Inverted Image")

    bright_img = trans.adjust_brighness(img)
    show_image(np.array(bright_img), title="Brightness Adjusted")

    contrast_img = trans.acjust_constract(img)
    show_image(np.array(contrast_img), title="Contrast Adjusted")

    rotated_img = trans.Rotate(img)
    show_image(np.array(rotated_img), title="Rotated Image")

    flipped_h = trans.Flip_Horizontal(img)
    show_image(np.array(flipped_h), title="Flipped Horizontal")

    flipped_v = trans.Flip_Vertical(img)
    show_image(np.array(flipped_v), title="Flipped Vertical")

    cropped_img = trans.Crop(img)
    show_image(np.array(cropped_img), title="Cropped Image")


    ai = AI_Features()
    blurred_img = ai.blur(img)
    show_image(np.array(blurred_img), title="Blurred Image")

    sharpened_img = ai.sharpen(img)
    show_image(np.array(sharpened_img), title="Sharpened Image")

    smoothed_img = ai.smooth(img)
    show_image(np.array(smoothed_img), title="Smoothed Image")
    
    print("All operations completed!")

if __name__ == "__main__":
    main()

